In [1]:
import pandas as pd 
import numpy as np

In [2]:
data = pd.read_csv('/Users/benjaminliang/Desktop/may3-4scrapyscrapefixednumbers.csv', encoding='utf-8') #or 'windows-1252' if utf-8 errors
data = data.drop_duplicates(keep='first', subset=['PMID'])
data.head()


,Indicator,MoA,pubmedurls_Article_Title,pubmedurls_Article_URL,pubmedurls_Article_Authors,PMID,pubmedurls_Article_Misc
0,Methamphetamine use disorder/methamphetamine a...,agonist,Safety tolerability of oral lisdexamfetamine i...,https://pubmed.ncbi.nlm.nih.gov/34006547/,"Ezard N, Clifford B, Dunlop A, Bruno R, Carr A...",34006547.0,BMJ Open. 2021 May 18;11(5:e044696. doi: 10.11...
1,Methamphetamine use disorder/methamphetamine a...,agonist,The 5-HT2C receptor as a therapeutic target fo...,https://pubmed.ncbi.nlm.nih.gov/33929084/,"Campbell EJ, Bonomo Y, Pastor A, Collins L, No...",33929084.0,Pharmacol Res Perspect. 2021 May;9(3:e00767. d...
2,Methamphetamine use disorder/methamphetamine a...,agonist,Subjective physiological effects of acute intr...,https://pubmed.ncbi.nlm.nih.gov/21072503/,"Rush CR, Stoops WW, Lile JA, Glaser PE, Hays LR.",21072503.0,Psychopharmacology (Berl. 2011 Apr;214(3:665-7...
3,Methamphetamine use disorder/methamphetamine a...,agonist,Varenicline treatment for methamphetamine depe...,https://pubmed.ncbi.nlm.nih.gov/29860057/,"Briones M, Shoptaw S, Cook R, Worley M, Swanso...",29860057.0,Drug Alcohol Depend. 2018 Aug 1;189:30-36. doi...
4,Methamphetamine use disorder/methamphetamine a...,agonist,Physiological subjective effects of acute intr...,https://pubmed.ncbi.nlm.nih.gov/21802442/,"Rush CR, Stoops WW, Lile JA, Glaser PE, Hays LR.",21802442.0,Pharmacol Biochem Behav. 2011 Nov;100(1:40-7. ...


In [3]:
len(data)

9414

In [3]:
from pymed import PubMed
from bs4 import BeautifulSoup
import requests
import re
import math

pubmed = PubMed(tool="macOS VSCode", email="beneric1232003@gmail.com")

In [4]:
import csv 
with open('abs_may3_may4.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Indicator", "MoA", "Title", "URL", "Author", "PMID", "Abstract", "Trial ID", "Phase", "Date"])
    i = 0
    currMoa = "None"
    for index, row in data.iterrows(): 
        print(i)
        #PMID 
        ID = str(row['PMID'])[:-2]
        print(ID)
        
        #Indicator
        indicator = row["Indicator"]
        
        #Abstract 
        try:
            article = pubmed.query(ID)
            article_content = next(article)
            abstract = article_content.abstract
            title = article_content.title
        except:
            abstract = "None"
            title = "None"
            print("Could not find abstract for: " + ID)
        
        #Mesh Terms and URL
        terms_list = []
        try:
            url = row["pubmedurls_Article_URL"]
            pmid = ID
            req = requests.get(url)
            soup = BeautifulSoup(req.content, "html.parser")
            
            mesh_section = soup.find("div", attrs={'id': 'mesh-terms', 'class': 'mesh-terms keywords-section'})
            keywords_list = mesh_section.find("ul", attrs={'class': 'keywords-list'})
            li_list = keywords_list.find_all("li")
            for li_indiv in li_list:
                try:
                    text_div = li_indiv.find("div", attrs={'class': 'keyword-actions dropdown-block'})
                    button_div = text_div.find("button")
                    #print(button_div.text)
                    button_text_raw = button_div.text
                    button_text_cleaned = button_text_raw.strip()
                    terms_list.append(button_text_cleaned)
                except:
                    pass
        except:
            print("Could not find mesh terms for: " + ID)
        abstract+=' '.join(terms_list)
    
        #MoA
        moa_list = ["partial", "antagonist", "inverse", "inhibitor", "activator", "stimulant", "modulator", "replacement", "antimetabolite", "blocker", "inducer", "agonist"]
        """
        if isinstance(row["MoA"], str):
            moa = row["MoA"]
            currMoa = moa
        else: 
            moa = currMoa
        """
        # raw_search_url = row["pubmedurls_link"].lower()
        # actual_moa = "None"
        # for moa in moa_list:
        #         if moa in raw_search_url:
        #             if moa == "partial":
        #                 actual_moa = "partial agonist"
        #             elif moa == "inverse":
        #                 actual_moa = "inverse agonist"
        #             else:
        #                 actual_moa = moa
        #             break
        actual_moa = row["MoA"]
        
        #Title and Authors
        try:
            title = row["pubmedurls_Article_Title"]
            authors = row["pubmedurls_Article_Authors"]
        except:
            title = "None"
            authors = "None"
        
        #date 
        try:
            raw_data = row["pubmedurls_Article_Misc"]
            dates_regex = "(\d\d\d\d.*?);"
            dates = re.findall(f"{dates_regex}", raw_data)[0]
        except:
            dates = "None"
        
        #phase
        try:
            req = requests.get(url)
            soup = BeautifulSoup(req.content, "html.parser")
            link = soup.find("a", attrs={'title': 'See in ClinicalTrials.gov'})
            raw_text = link.text
            trial_id = raw_text.strip()
            URL = "https://clinicaltrials.gov/ct2/show/" + trial_id
            try: 
                page = requests.get(URL)
                soup = BeautifulSoup(page.content, "html.parser")
                table_contents = soup.find_all("span", attrs={'style': 'display:block;margin-bottom:1ex;'})
                phase = table_contents[-1].text
            except:
                phase = "Not Found"
        except:
            trial_id = "None"
            phase = "Not Found"
        if phase.find("Phase") == -1:
            phase = "Not Found"     

        
        #write
        writer.writerow([indicator, actual_moa, title, url, authors, pmid, abstract, trial_id, phase, dates])
        i+=1

0
34006547
1
33929084
2
21072503
3
29860057
4
21802442
5
23186131
6
24393456
7
29028556
8
19058926
9
27905916
10
33497547
11
35605529
12
35438671
13
31820289
14
26468683
15
31812878
16
30266003
17
26674372
18
15850499
19
22221594
20
22065532
21
27312405
22
28734107
23
10731625
24
16448579
25
24464531
26
19698734
27
15452681
28
36788634
29
30526230
30
16721698
31
25801501
32
28160494
33
17470315
34
32220414
35
26993372
36
15022183
37
20655182
38
15925741
39
29064909
40
9426992
41
10435398
42
12097810
43
25622123
44
10581644
45
16966620
46
15543630
47
22230648
48
27269956
49
20493549
50
22269534
51
17581531
52
24894963
53
31823620
54
1794016
55
35488084
56
18207225
57
31362784
58
18248689
59
16413604
60
16621339
61
23297867
62
31042696
63
9330778
64
21592681
65
11672493
66
25818061
67
12921918
68
21624391
69
9373724
70
6777463
71
21803113
72
3087968
73
15136987
74
14628189
75
22534658
76
19646280
77
22070720
78
8748632
79
12472171
80
9438680
81
31825466
82
36265196
83
37004463
84
3506808